In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

100.0%


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100.0%


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100.0%


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100.0%

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [2]:
loss_fn = nn.CrossEntropyLoss()

In [3]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

NameError: name 'learning_rate' is not defined

In [11]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)
        sum_loss += loss.item()

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [19]:
def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return correct, test_loss

In [48]:
def run_training(epochs, train_dataloader, model, loss_fn, optimizer):
    test_loss = []
    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        sum_loss = train_loop(train_dataloader, model, loss_fn, optimizer)
        train_loss.append(sum_loss)

        acc, avg_loss = test_loop(test_dataloader, model, loss_fn)
        test_loss.append(avg_loss)
    print("Done!")
    
    return test_loss

In [49]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [50]:
test_loss_sgd = run_training(epochs, train_dataloader, model, loss_fn, optimizer)

Epoch 1
-------------------------------
loss: 0.270989  [   64/60000]
loss: 0.362948  [ 6464/60000]
loss: 0.229244  [12864/60000]
loss: 0.379360  [19264/60000]
loss: 0.319152  [25664/60000]
loss: 0.353176  [32064/60000]
loss: 0.304867  [38464/60000]
loss: 0.418580  [44864/60000]
loss: 0.442949  [51264/60000]
loss: 0.347890  [57664/60000]
Test Error: 
 Accuracy: 85.9%, Avg loss: 0.389633 

Epoch 2
-------------------------------
loss: 0.221511  [   64/60000]
loss: 0.311758  [ 6464/60000]
loss: 0.209710  [12864/60000]
loss: 0.364977  [19264/60000]
loss: 0.308115  [25664/60000]
loss: 0.346503  [32064/60000]
loss: 0.306525  [38464/60000]
loss: 0.421079  [44864/60000]
loss: 0.430594  [51264/60000]
loss: 0.335855  [57664/60000]
Test Error: 
 Accuracy: 86.1%, Avg loss: 0.383205 

Epoch 3
-------------------------------
loss: 0.213693  [   64/60000]
loss: 0.306204  [ 6464/60000]
loss: 0.202028  [12864/60000]
loss: 0.355419  [19264/60000]
loss: 0.307100  [25664/60000]
loss: 0.345129  [32064/600

In [51]:
print(test_loss_sgd)

[0.38963310667283974, 0.38320457574668204, 0.37977952429443407, 0.37754955802373824, 0.3758986291422206]


In [52]:
model = NeuralNetwork()
optimizer = torch.optim.RMSprop(model.parameters(), lr = learning_rate, alpha = 0.9)
test_loss_rms = run_training(epochs, train_dataloader, model, loss_fn, optimizer)

Epoch 1
-------------------------------
loss: 2.299017  [   64/60000]
loss: 0.561404  [ 6464/60000]
loss: 0.464267  [12864/60000]
loss: 0.482081  [19264/60000]
loss: 0.479731  [25664/60000]
loss: 0.463993  [32064/60000]
loss: 0.374960  [38464/60000]
loss: 0.521036  [44864/60000]
loss: 0.493906  [51264/60000]
loss: 0.431683  [57664/60000]
Test Error: 
 Accuracy: 82.5%, Avg loss: 0.466934 

Epoch 2
-------------------------------
loss: 0.275227  [   64/60000]
loss: 0.337481  [ 6464/60000]
loss: 0.290373  [12864/60000]
loss: 0.337556  [19264/60000]
loss: 0.343578  [25664/60000]
loss: 0.383674  [32064/60000]
loss: 0.286450  [38464/60000]
loss: 0.419908  [44864/60000]
loss: 0.446033  [51264/60000]
loss: 0.404067  [57664/60000]
Test Error: 
 Accuracy: 85.2%, Avg loss: 0.415195 

Epoch 3
-------------------------------
loss: 0.222330  [   64/60000]
loss: 0.343819  [ 6464/60000]
loss: 0.214913  [12864/60000]
loss: 0.317007  [19264/60000]
loss: 0.323751  [25664/60000]
loss: 0.411305  [32064/600

In [53]:
test_loss_rms

[0.4669338763709281,
 0.4151948234837526,
 0.3936103121110588,
 0.4195809263712282,
 0.44202277605321]

In [ ]:
# Постройте графики зависимости сходимости (значение функции потерь от количества итераций работы алгоритма)
# от выбора оптимизационного метода
# количество эпох = 50
# Сравните, сделайте выводы
# https://pytorch.org/docs/stable/optim.html